In [4]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

def generate_scale_free_network(N, alpha):
    k_min = 1
    k_max = int(np.sqrt(N))
    degrees = []
    while len(degrees) < N:
        k = np.random.zipf(alpha)
        if k_min <= k <= k_max:
            degrees.append(k)
    if sum(degrees) % 2 == 1:
        degrees[0] += 1
    G = nx.configuration_model(degrees)
    G = nx.Graph(G)
    G.remove_edges_from(nx.selfloop_edges(G))
    return G

def rewire_edges_to_target_assortativity(G, target_rkk, max_iter=50000):
    current_rkk = nx.degree_assortativity_coefficient(G)
    iterations = 0
    edges = list(G.edges())
    while abs(current_rkk - target_rkk) > 0.01 and iterations < max_iter:
        idx1, idx2 = np.random.choice(len(edges), 2, replace=False)
        e1 = edges[idx1]
        e2 = edges[idx2]
        if len(set(e1 + e2)) == 4:
            G_temp = G.copy()
            G_temp.remove_edge(*e1)
            G_temp.remove_edge(*e2)
            G_temp.add_edge(e1[0], e2[1])
            G_temp.add_edge(e2[0], e1[1])
            new_rkk = nx.degree_assortativity_coefficient(G_temp)
            if abs(new_rkk - target_rkk) < abs(current_rkk - target_rkk):
                G.remove_edge(*e1)
                G.remove_edge(*e2)
                G.add_edge(e1[0], e2[1])
                G.add_edge(e2[0], e1[1])
                edges = list(G.edges())
                current_rkk = new_rkk
        iterations += 1
    return G

def assign_attributes_with_corr(G, px1, target_rho_kx, max_swaps=100000):
    N = G.number_of_nodes()
    degrees = np.array([d for n, d in G.degree()])
    attributes = np.zeros(N, dtype=int)
    active_indices = np.random.choice(N, int(px1*N), replace=False)
    attributes[active_indices] = 1
    def corr_d_x():
        return pearsonr(degrees, attributes)[0]
    corr = corr_d_x()
    swaps = 0
    while abs(corr - target_rho_kx) > 0.01 and swaps < max_swaps:
        idx1 = np.random.choice(np.where(attributes == 1)[0])
        idx0 = np.random.choice(np.where(attributes == 0)[0])
        current_corr = corr
        attributes[idx1], attributes[idx0] = attributes[idx0], attributes[idx1]
        new_corr = corr_d_x()
        if abs(new_corr - target_rho_kx) < abs(current_corr - target_rho_kx):
            corr = new_corr
        else:
            attributes[idx1], attributes[idx0] = attributes[idx0], attributes[idx1]
        swaps += 1
    return attributes

def majority_illusion_fraction(G, attributes):
    count = 0
    for n in G.nodes():
        neighbors = list(G.neighbors(n))
        if len(neighbors) == 0:
            continue
        active_neighbors = sum(attributes[nb] for nb in neighbors)
        if active_neighbors > len(neighbors)/2:
            count += 1
    return count / G.number_of_nodes()

# Parámetros ajustados: rkk diferentes para cada alpha según paper
N = 2000
px1 = 0.05
params = {
    2.1: [-0.4, -0.1, 0.1, 0.3],
    2.4: [-0.45, -0.15, 0.05, 0.15],
    3.1: [-0.6, -0.25, 0.05, 0.25]
}

results = {}
for alpha, rkk_list in params.items():
    results[alpha] = {}
    G = generate_scale_free_network(N, alpha)
    for rkk_target in rkk_list:
        G_rewired = rewire_edges_to_target_assortativity(G.copy(), rkk_target)
        rho_kxs = np.linspace(0, 0.6, 10)
        results[alpha][rkk_target] = []
        for rho_kx in rho_kxs:
            attrs = assign_attributes_with_corr(G_rewired, px1, rho_kx)
            frac = majority_illusion_fraction(G_rewired, attrs)
            results[alpha][rkk_target].append((rho_kx, frac))
            print(f'alpha={alpha}, rkk={rkk_target:.2f}, rho_kx={rho_kx:.2f} --> frac={frac:.3f}')

plt.figure(figsize=(12, 7))
colors = ['blue', 'orange', 'green', 'red']
markers = ['o', 's', '^', 'D']

for alpha, rkk_list in params.items():
    for idx, rkk_target in enumerate(rkk_list):
        data = results[alpha][rkk_target]
        x = [d[0] for d in data]
        y = [d[1] for d in data]
        linestyle = '-' if alpha == 2.1 else '--' if alpha == 2.4 else ':'
        plt.plot(x, y, label=f'$\\alpha={alpha}, r_{{kk}}={rkk_target}$',
                 color=colors[idx], marker=markers[idx], linestyle=linestyle)

plt.xlabel('Degree-attribute correlation $\\rho_{kx}$')
plt.ylabel('Fraction of nodes with majority active neighbors')
plt.title('Majority Illusion in Scale-Free Networks (Fig. 2 style) with varied $r_{kk}$ per $\\alpha$')
plt.legend(title='Parameters', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


alpha=2.1, rkk=-0.40, rho_kx=0.00 --> frac=0.036
alpha=2.1, rkk=-0.40, rho_kx=0.07 --> frac=0.054
alpha=2.1, rkk=-0.40, rho_kx=0.13 --> frac=0.092
alpha=2.1, rkk=-0.40, rho_kx=0.20 --> frac=0.131
alpha=2.1, rkk=-0.40, rho_kx=0.27 --> frac=0.155
alpha=2.1, rkk=-0.40, rho_kx=0.33 --> frac=0.177
alpha=2.1, rkk=-0.40, rho_kx=0.40 --> frac=0.228
alpha=2.1, rkk=-0.40, rho_kx=0.47 --> frac=0.266
alpha=2.1, rkk=-0.40, rho_kx=0.53 --> frac=0.308
alpha=2.1, rkk=-0.40, rho_kx=0.60 --> frac=0.350
alpha=2.1, rkk=-0.10, rho_kx=0.00 --> frac=0.030
alpha=2.1, rkk=-0.10, rho_kx=0.07 --> frac=0.052
alpha=2.1, rkk=-0.10, rho_kx=0.13 --> frac=0.077
alpha=2.1, rkk=-0.10, rho_kx=0.20 --> frac=0.083
alpha=2.1, rkk=-0.10, rho_kx=0.27 --> frac=0.105
alpha=2.1, rkk=-0.10, rho_kx=0.33 --> frac=0.131
alpha=2.1, rkk=-0.10, rho_kx=0.40 --> frac=0.150
alpha=2.1, rkk=-0.10, rho_kx=0.47 --> frac=0.181
alpha=2.1, rkk=-0.10, rho_kx=0.53 --> frac=0.193
alpha=2.1, rkk=-0.10, rho_kx=0.60 --> frac=0.227
alpha=2.1, rkk=0.10,

KeyboardInterrupt: 